# Import Libraries

In [460]:
import sys
sys.path.append("/home/felix/PycharmProjects/Quantum-Challenge/")
import pandas as pd
import xarray as xr
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import traceback
import math

import networkx as nx
import itertools

from qiskit.algorithms import AmplificationProblem
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.algorithms import Grover
from qiskit.circuit.library import GroverOperator
from qiskit.extensions import Initialize

# importing Qiskit
from qiskit import IBMQ, Aer, assemble, transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy

# import basic plot tools
from qiskit.visualization import plot_histogram

from itertools import product 
from qiskit.quantum_info import Statevector

from importlib import reload
import utils.utils as ut

# Keywords

Classical approach: selection of the trajectory with the shortest Euclidean distance to the target in order to minimize the distance traveled, i.e. save fuel and time

try to find argument on how many planes can fit into single voxel with

# TODO

- Two seperate Quantum Models: Grovers search algorithm to find a climate optimized trajectory and FVQE to de-conflict computed trajectories. 


- Simulate plane trajec with timestamps in mp4 (animated plot)
- Argue for discretization i.e. continuus optimal control problem cannot be solved easitly with current qubit number, therefore disc is used to simplify and then interpolated to regenerate the traj in the end. THe interpolation assumes smooth trajectories with neglegible delta climate impact compared to the disc traj.
- De-conflicitng

# Load flight Data 

In [461]:
flight_df = pd.read_csv("../data/flights.csv", sep=";")

# Load velocity and fuel consumption data

In [462]:
cruise_df = pd.read_pickle("../data/cruise_df.pkl")
climb_df = pd.read_pickle("../data/climb_df.pkl")
descent_df = pd.read_pickle("../data/descent_df.pkl")

# Load climate data

In [463]:
climate_df = pd.read_pickle("../data/climate_df.pkl")

# Load Climate Optimized Flight Schedule

In [464]:
q_traj_arr = np.load("../data/c_climate_trajectory_arr.npy", allow_pickle=True)

# De-Conflicting

List of trajecs with overlap 
each qubit is a decision if point of conflict is resolved by +-20 FL 
The structure needed for that is the traj, pos of conflict in the trajectory and the list of all trajecs

Resolving is done by looking at direc of travel and the introdocung either (x+-1,y,z+-1) or (x,y+-1,z+-1)



In [488]:
reload(ut)

<module 'utils.utils' from '/home/felix/PycharmProjects/Quantum-Challenge/utils/utils.py'>

In [489]:
c, overlap_dict = ut.constraint_n_planes(q_traj_arr)

In [470]:

confl_list = np.array(list(overlap_dict.keys()))
choice_inx = np.random.choice(np.arange(len(confl_list)),size=5)
confl_arr = confl_list[choice_inx]
confl_arr

array([[22, 24],
       [47, 24],
       [16, 11],
       [18, 15],
       [58, 19]])

In [498]:
flight_df["cost"] = [ut.C(traj) for traj in q_traj_arr]
flight_df

flight_number start_time  start_flightlevel  start_longitudinal  \
0               0   07:15:00                250                 -30   
1               1   07:10:00                310                 -30   
2               2   07:50:00                250                 -22   
3               3   07:30:00                260                 -26   
4               4   07:00:00                230                 -30   
..            ...        ...                ...                 ...   
94             95   06:10:00                260                 -30   
95             96   07:05:00                200                 -30   
96             97   06:00:00                300                 -14   
97             98   07:30:00                310                 -30   
98             99   06:00:00                340                 -18   

    start_latitudinal  end_longitudinal  end_latitudinal          cost  
0                  56                24               60  3.092848e-08  
1                  56                30               42  5.424868e-08  
2                  34                30               46  4.734343e-08  
3                  34                28               60  7.150705e-08  
4                  42                30               46  4.596743e-08  
..                ...               ...              ...           ...  
94                 40                26               60  5.849001e-08  
95                 42                22               60  5.477746e-08  
96                 34                30               52  4.471082e-08  
97                 40                30               56  6.343778e-08  
98                 34                28               60  5.498489e-08  

[99 rows x 8 columns]

In [499]:
reload(ut)
ut.vqe_cost((0,0,0,0,0), confl_arr, res_q_sim, flight_df)

7.878172675197086e-06

In [403]:
import math
import numpy as np
import matplotlib.pyplot as plt
from pytket.circuit import Circuit
from pytket.extensions.qiskit import AerStateBackend, AerBackend  
from pytket.circuit.display import render_circuit_jupyter
from pytket.utils.results import probs_from_state  
from qiskit.quantum_info import partial_trace, entropy, Statevector  


In [500]:
def ansatz(params):
    c = Circuit(n_qubits,0)
    params = params / np.pi  # pytket works in units of pi: Ry(p, q) = exp(-i pi p/2 Y_q)    c = Circuit(n_qubits)
    param_index = 0
    for q in range(n_qubits):
        c.Ry(params[param_index], q)
        param_index += 1
    for l in range(n_layers):
        for q in range(n_qubits-1):
            c.CX(q, q+1)
        for q in range(n_qubits):
            c.Ry(params[param_index], q)
            param_index += 1
    return c

n_qubits = 5
n_layers = 1  # number of ansatz circuit layers
n_params = n_qubits * (1 + n_layers)  

In [501]:
render_circuit_jupyter(ansatz(np.ones(n_params) * np.pi))

In [502]:
# Filter: function f: c real to f(c) real such that f^2(c) strictly decreases

t = 1  # t>0 is a function parameter. Could be changed at each optimization step

def f(c, t):
    return np.exp(-t * c)

In [503]:
# Sample ansatz circuit with tKet

# number of measurement shots. Each shot is one sample of the bitstring distribution produced by the quantum ansatz
n_shots = 1
backend = AerBackend()

def sample_ansatz(params):
    c = ansatz(params)
    c.measure_all()
    backend.compile_circuit(c)
    handle = backend.process_circuit(c, n_shots=n_shots)
    result = backend.get_result(handle)
    counts = result.get_counts()
    probs = {bitstring: c / n_shots for bitstring, c in counts.items()}  # samples from the bitstring distribution
    return probs

In [504]:
# Evaluate filter expectation value and build up the spectrum

spectrum = {}  # will contain a pair (bitstring, C(bitstring)) for each different sampled bitstring during experiment

def evaluate(probs, t, spectrum):
    filter_exp = 0
    for bitstring, p in probs.items():
        if bitstring not in spectrum:
            cost = vqe_cost(bitstring, confl_arr, res_q_sim)
            spectrum[bitstring] = cost
        else:
            cost = spectrum[bitstring]
        filter_exp += f(cost, t) * p
    return filter_exp, spectrum

In [505]:
# Gradient evaluated at params

def g(params, t, spectrum):
    output = np.zeros(n_params)
    for i in range(n_params):
        params_plus = params + np.bincount([i], None, n_params) * np.pi / 2
        params_minus = params - np.bincount([i], None, n_params) * np.pi / 2
        probs_plus = sample_ansatz(params_plus)
        probs_minus = sample_ansatz(params_minus)
        filter_exp_plus, spectrum = evaluate(probs_plus, t, spectrum)
        filter_exp_minus, spectrum = evaluate(probs_minus, t, spectrum)
        output[i] = filter_exp_minus - filter_exp_plus
    return output, spectrum

In [506]:
# Gradient-based optimiser

learning_rate = 0.01

def update_params(params, gradient):
    return params - learning_rate * gradient

In [507]:
# F-VQE algorithm
    
# initialises the quantum computer in the equal superposition

def fvqe(params, n_steps):
    spectrum = {}
    for step in range(n_steps):
        gradient, spectrum = g(params, t, spectrum)
        params = update_params(params, gradient)
        best_cost = min(spectrum.values())
        best_solutions = {bitstring for bitstring, cost in spectrum.items() if cost==best_cost}
        print(f"step {step}: best cost = {best_cost} for {best_solutions}")
    return spectrum

In [508]:
params = np.array(list(np.zeros(n_params - n_qubits)) + list(np.ones(n_qubits) * np.pi / 2)) 

fvqe(params, n_steps = 3)

step 0: best cost = 7.878024603172714e-06 for {(0, 0, 1, 0, 1), (0, 0, 0, 0, 1), (1, 1, 0, 1, 1), (0, 1, 1, 0, 1), (1, 0, 1, 0, 1)}
step 1: best cost = 7.878024603172714e-06 for {(0, 0, 1, 0, 1), (0, 1, 0, 1, 1), (0, 0, 1, 1, 1), (0, 1, 0, 0, 1), (0, 0, 0, 0, 1), (1, 0, 0, 1, 1), (1, 1, 0, 1, 1), (0, 0, 0, 1, 1), (0, 1, 1, 0, 1), (1, 1, 1, 0, 1), (1, 0, 1, 0, 1)}
step 2: best cost = 7.878024603172714e-06 for {(0, 0, 1, 0, 1), (1, 0, 1, 1, 1), (0, 1, 0, 1, 1), (0, 0, 1, 1, 1), (0, 1, 0, 0, 1), (1, 0, 0, 0, 1), (0, 0, 0, 0, 1), (1, 0, 0, 1, 1), (1, 1, 0, 1, 1), (0, 0, 0, 1, 1), (0, 1, 1, 0, 1), (0, 1, 1, 1, 1), (1, 1, 1, 0, 1), (1, 0, 1, 0, 1)}


{(1, 0, 1, 1, 0): 7.878172675197086e-06,
 (0, 1, 0, 1, 0): 7.878172675197086e-06,
 (0, 0, 1, 1, 0): 7.878172675197086e-06,
 (0, 0, 0, 0, 1): 7.878024603172714e-06,
 (1, 0, 0, 0, 0): 7.878172675197086e-06,
 (0, 0, 1, 0, 1): 7.878024603172714e-06,
 (0, 0, 1, 0, 0): 7.878172675197086e-06,
 (1, 1, 0, 1, 1): 7.878024603172714e-06,
 (0, 1, 1, 0, 0): 7.878172675197086e-06,
 (1, 0, 1, 0, 1): 7.878024603172714e-06,
 (0, 1, 1, 0, 1): 7.878024603172714e-06,
 (1, 0, 0, 1, 0): 7.878172675197086e-06,
 (0, 0, 0, 0, 0): 7.878172675197086e-06,
 (0, 1, 0, 0, 0): 7.878172675197086e-06,
 (1, 1, 1, 1, 0): 7.878172675197086e-06,
 (0, 1, 0, 0, 1): 7.878024603172714e-06,
 (1, 1, 1, 0, 1): 7.878024603172714e-06,
 (0, 0, 0, 1, 1): 7.878024603172714e-06,
 (1, 0, 0, 1, 1): 7.878024603172714e-06,
 (1, 0, 1, 0, 0): 7.878172675197086e-06,
 (0, 1, 0, 1, 1): 7.878024603172714e-06,
 (0, 0, 1, 1, 1): 7.878024603172714e-06,
 (1, 1, 0, 1, 0): 7.878172675197086e-06,
 (0, 1, 1, 1, 1): 7.878024603172714e-06,
 (1, 1, 1, 0, 0)